<a href="https://colab.research.google.com/github/Alfredo2212/ds-takehome/blob/main/notebooks/B_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


%matplotlib inline
sns.set(style="whitegrid")
warnings.filterwarnings('ignore')

In [10]:
# import dataset
url = "https://raw.githubusercontent.com/Alfredo2212/ds-takehome/main/data/credit_scoring.csv"
df = pd.read_csv(url)

In [11]:
df.head()

,application_id,age,monthly_income,loan_amount,previous_defaults,credit_score,default,leak_col_good,leak_col_subtle
0,501000,41,13995609,5982664,0,624,0,0,-0.040
1,501001,58,13683833,3711198,0,809,0,0,0.001
2,501002,33,9417391,7172332,0,647,0,0,0.077
3,501003,45,6861811,8661056,0,450,0,0,0.038
4,501004,22,5640742,4520669,1,816,0,0,0.020


In [12]:
# Place leak cols into df_leak
leak_cols = ['leak_col_good','leak_col_subtle']
df_leak = df[leak_cols].copy()

# Drop Application ID & leak_cols from main dataframe
df = df.drop(columns=leak_cols + ['application_id'])


In [13]:
df_leak.head()

,leak_col_good,leak_col_subtle
0,0,-0.040
1,0,0.001
2,0,0.077
3,0,0.038
4,0,0.020


In [14]:
# Check null values for df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   age                6000 non-null   int64
 1   monthly_income     6000 non-null   int64
 2   loan_amount        6000 non-null   int64
 3   previous_defaults  6000 non-null   int64
 4   credit_score       6000 non-null   int64
 5   default            6000 non-null   int64
dtypes: int64(6)
memory usage: 281.4 KB


In [7]:
# Check duplicate valeus for df
df.duplicated().sum()

np.int64(0)

There is no null and duplicate values, dataset is ready for EDA.

In [15]:
# Check data distribution, skewness and descriptive statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,6000.0,4.032150e+01,1.130070e+01,21.0,31.00,41.0,50.00,59.0
monthly_income,6000.0,1.140386e+07,4.912598e+06,3000310.0,7163379.00,11299453.0,15737051.75,19986217.0
loan_amount,6000.0,8.001122e+06,4.071786e+06,1000794.0,4440939.00,7982303.5,11531445.00,14998580.0
previous_defaults,6000.0,2.436667e-01,4.847670e-01,0.0,0.00,0.0,0.00,3.0
credit_score,6000.0,5.759798e+02,1.583289e+02,300.0,438.75,577.5,713.00,849.0
default,6000.0,1.216667e-02,1.096387e-01,0.0,0.00,0.0,0.00,1.0


Descriptive statistics :
Age - limited age range ( 21 - 59 ), this suggests that individuals beyond this range are excluded from loan eligibility.
Monthly Income & Loan Amount - Very high variance in millions, skewed distributions.
Previous Defaults - Most values are 0, this informs imbalanced distribution.
Default : Dependent variable, with 1.2% default rate ( Mean 1.21 e-02 )